<a href="https://colab.research.google.com/github/oytisa/etspatial/blob/main/AnyGeospatial_Image_Segmentation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Discover the potential of Meta AI’s Segment Anything Model (SAM) in this comprehensive tutorial. We dive into SAM, an efficient and promptable model for image segmentation. **

In [ ]:
# Install dependencies
%pip install segment-geospatial leafmap localtileserver

In [ ]:
import os
import leafmap
import torch
from samgeo import SamGeo, tms_to_geotiff

In [ ]:
m = leafmap.Map(center=[7.10049, 38.62629], zoom=19)
m.add_basemap('SATELLITE')
m

In [ ]:
# Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map
bbox = m.user_roi_bounds()
print(bbox)

In [ ]:
image = 'satellite2.tif'
# image = '/path/to/your/own/image.tif'

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source='Satellite', overwrite=True)

In [ ]:
m.add_raster(image, layer_name='Image')
m

In [ ]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

In [ ]:
#Segment the image
mask = 'segment.tiff'
sam.generate(image, mask)

In [ ]:
#save as geopackage
vector = 'segment.gpkg'
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

In [ ]:
#if to save as shapefile
shapefile = 'segment.shp'
sam.tiff_to_vector(mask, shapefile)

In [ ]:
#visualize the segmentation result
style = {
    'color': '#3388ff',
    'weight': 2,
    'fillColor': '#7c4185',
    'fillOpacity': 0.5,
}
m.add_vector(vector, layer_name='Vector', style=style)
m